In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression

In [2]:
train_data = pd.read_csv('train_data.csv', usecols=['stemmed_tokens', 'type'])                            # Reads train_data csv file created earlier to DataFrame
validation_data = pd.read_csv('validation_data.csv', usecols=['stemmed_tokens', 'type'])                  # Reads validation_data csv file created earlier to DataFrame

# Extract content and labels
X_train = train_data['stemmed_tokens']
y_train = train_data['type']
X_val = validation_data['stemmed_tokens']
y_val = validation_data['type']

In [3]:
# Convert text to TF-IDF features (tager sådan en halv time tror jeg )
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1,2))  # Use unigrams & bigrams
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)

In [4]:
# Train logistic regression model (forvent omkring 86 f1)
model = LogisticRegression(max_iter=1500, class_weight='balanced') 
model.fit(X_train_tfidf, y_train)

# Prediction
y_val_pred = model.predict(X_val_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred, pos_label=1)
report = classification_report(y_val, y_val_pred)

# Print results
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print("Classification Report:\n", report)

Accuracy: 0.8683
F1 Score: 0.8631
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.87      0.87     39216
           1       0.86      0.87      0.86     35770

    accuracy                           0.87     74986
   macro avg       0.87      0.87      0.87     74986
weighted avg       0.87      0.87      0.87     74986



## GridSearchCV for Optimizing Logistic Regression Performance

In [ ]:
from sklearn.experimental import enable_halving_search_cv  # Import this first
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np



# Ensure y_train is dense
y_train_dense = y_train.toarray() if hasattr(y_train, "toarray") else y_train

# Ensure X_train_tfidf is dense
X_train_dense = X_train_tfidf.toarray() if hasattr(X_train_tfidf, "toarray") else X_train_tfidf
# Example parameter grid
param_grid = [
    {'penalty':['l1','l2','elasticnet','none'],
    'C' : np.logspace(-4,4,20),
    'solver': ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter'  : [100,1000,2500,5000]
}
]
# Instantiate HalvingRandomSearchCV
search = HalvingRandomSearchCV(
    estimator=LogisticRegression(class_weight='balanced'),
    param_distributions=param_grid,
    cv=4,
    verbose=2,
    #min_resources=2  # Adjust min_resources to be smaller or equal to the total number of samples
)





# Fit the model
search.fit(X_train_dense, y_train_dense)

# Get the best model
best_model = search.best_estimator_

# Convert search results to DataFrame
results_df = pd.DataFrame(search.cv_results_)

# Select relevant columns
columns_to_display = ["param_C", "param_max_iter", "mean_test_score", "rank_test_score", "iter"]
results_df = results_df[columns_to_display]

# Sort by best rank
results_df = results_df.sort_values(by="rank_test_score")

# Print top results
print(results_df.head())

n_iterations: 7
n_required_iterations: 7
n_possible_iterations: 10
min_resources_: 16
max_resources_: 599891
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 1600
n_resources: 16
Fitting 4 folds for each of 1600 candidates, totalling 6400 fits
[CV] END ...C=0.0001, max_iter=100, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...C=0.0001, max_iter=100, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...C=0.0001, max_iter=100, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...C=0.0001, max_iter=100, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END C=0.0001, max_iter=100, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END C=0.0001, max_iter=100, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END C=0.0001, max_iter=100, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END C=0.0001, max_iter=100, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END C=0.0001, max_iter=100, penalty=l1, solver=liblinear; total time=   0.0

/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 1600 is smaller than n_iter=37493. Running 1600 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[CV] END ...C=0.0001, max_iter=1000, penalty=l1, solver=saga; total time=   0.0s
[CV] END ...C=0.0001, max_iter=1000, penalty=l1, solver=saga; total time=   0.0s
[CV] END ...C=0.0001, max_iter=1000, penalty=l1, solver=saga; total time=   0.0s
[CV] END ...C=0.0001, max_iter=1000, penalty=l1, solver=saga; total time=   0.0s
[CV] END ..C=0.0001, max_iter=1000, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END ..C=0.0001, max_iter=1000, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END ..C=0.0001, max_iter=1000, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END ..C=0.0001, max_iter=1000, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END C=0.0001, max_iter=1000, penalty=l2, solver=newton-cg; total time=   0.0s
[CV] END C=0.0001, max_iter=1000, penalty=l2, solver=newton-cg; total time=   0.0s
[CV] END C=0.0001, max_iter=1000, penalty=l2, solver=newton-cg; total time=   0.0s
[CV] END C=0.0001, max_iter=1000, penalty=l2, solver=newton-cg; total time=   0.0s
[CV] END C=0.0001, m

/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END C=4.281332398719396, max_iter=100, penalty=l1, solver=saga; total time=   0.1s
[CV] END C=4.281332398719396, max_iter=100, penalty=l1, solver=saga; total time=   0.0s
[CV] END C=4.281332398719396, max_iter=100, penalty=l1, solver=saga; total time=   0.1s
[CV] END C=4.281332398719396, max_iter=100, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END C=4.281332398719396, max_iter=100, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END C=4.281332398719396, max_iter=100, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END C=4.281332398719396, max_iter=100, penalty=l2, solver=lbfgs; total time=   0.0s
[CV] END C=4.281332398719396, max_iter=100, penalty=l2, solver=newton-cg; total time=   0.0s
[CV] END C=4.281332398719396, max_iter=100, penalty=l2, solver=newton-cg; total time=   0.0s
[CV] END C=4.281332398719396, max_iter=100, penalty=l2, solver=newton-cg; total time=   0.0s
[CV] END C=4.281332398719396, max_iter=100, penalty=l2, solver=newton-cg; total time=   0.0s
[CV] END

...

/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_search.py:1102: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.60416667 ... 0.77669096 0.7764335  0.7764335 ]
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_search.py:1102: UserWarning: One or more of the train scores are non-finite: [       nan        nan 0.39962121 ... 1.         1.         1.        ]
  warnings.warn(


[CV] END C=78.47599703514607, max_iter=5000, penalty=l2, solver=sag; total time=  41.6s
[CV] END C=78.47599703514607, max_iter=5000, penalty=l2, solver=sag; total time=  44.2s
[CV] END C=78.47599703514607, max_iter=5000, penalty=l2, solver=sag; total time=  45.2s
[CV] END C=78.47599703514607, max_iter=5000, penalty=l2, solver=sag; total time=  47.1s
[CV] END C=78.47599703514607, max_iter=5000, penalty=l2, solver=saga; total time= 1.1min
[CV] END C=78.47599703514607, max_iter=5000, penalty=l2, solver=saga; total time= 1.1min
[CV] END C=78.47599703514607, max_iter=5000, penalty=l2, solver=saga; total time= 1.2min
[CV] END C=78.47599703514607, max_iter=5000, penalty=l2, solver=saga; total time= 1.0min
[CV] END C=78.47599703514607, max_iter=1000, penalty=l2, solver=liblinear; total time=   1.1s
[CV] END C=78.47599703514607, max_iter=1000, penalty=l2, solver=liblinear; total time=   1.2s
[CV] END C=78.47599703514607, max_iter=1000, penalty=l2, solver=liblinear; total time=   1.2s
[CV] END C

/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_search.py:1102: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.60416667 ... 0.80451028 0.80476754 0.80433875]
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_search.py:1102: UserWarning: One or more of the train scores are non-finite: [       nan        nan 0.39962121 ... 0.99979994 0.99979994 0.99979994]
  warnings.warn(


: 

: 

In [ ]:
from sklearn.experimental import enable_halving_search_cv  # Import this first
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np

y_train_dense = y_train.toarray() if hasattr(y_train, "toarray") else y_train

# Ensure X_train_tfidf is dense
X_train_dense = X_train_tfidf.toarray() if hasattr(X_train_tfidf, "toarray") else X_train_tfidf

# Example parameter grid
param_grid = [
    {'penalty':['l1','l2', 'elasticnet'],
    'C' : np.logspace(-4,4,10),
    'solver': ['saga','liblinear'],
    'max_iter'  : [1000, 2500, 5000]
}
]

# Instantiate HalvingRandomSearchCV
search = HalvingRandomSearchCV(
    estimator=LogisticRegression(class_weight='balanced'),
    param_distributions=param_grid,
    cv=5,
    verbose=2,
    #min_resources=2  # Adjust min_resources to be smaller or equal to the total number of samples
)

# Fit the model
search.fit(X_train_dense, y_train_dense)

# # Get the best model
# best_model = search.best_estimator_

# Best Model Evaluation
y_val_pred = search.predict(X_train_dense)  # Directly using search to predict

accuracy = accuracy_score(y_train_dense, y_val_pred)
f1 = f1_score(y_train_dense, y_val_pred, average='weighted')
report = classification_report(y_train_dense, y_val_pred)

# Print best parameters and evaluation metrics
print("Best Parameters:", search.best_params_)
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print("Classification Report:\n", report)

# Convert search results to DataFrame
results_df = pd.DataFrame(search.cv_results_)

# Select relevant columns
columns_to_display = ["param_C", "param_max_iter", "mean_test_score", "rank_test_score", "iter"]
results_df = results_df[columns_to_display]

# Sort by best rank
results_df = results_df.sort_values(by="rank_test_score")

# Print top results
print(results_df.head())

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 10
min_resources_: 20
max_resources_: 599891
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 180
n_resources: 20
Fitting 5 folds for each of 180 candidates, totalling 900 fits
[CV] END ...C=0.0001, max_iter=1000, penalty=l1, solver=saga; total time=   0.0s
[CV] END ...C=0.0001, max_iter=1000, penalty=l1, solver=saga; total time=   0.0s
[CV] END ...C=0.0001, max_iter=1000, penalty=l1, solver=saga; total time=   0.0s
[CV] END ...C=0.0001, max_iter=1000, penalty=l1, solver=saga; total time=   0.0s
[CV] END ...C=0.0001, max_iter=1000, penalty=l1, solver=saga; total time=   0.0s
[CV] END C=0.0001, max_iter=1000, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END C=0.0001, max_iter=1000, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END C=0.0001, max_iter=1000, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END C=0.0001, max_iter=1000, penalty=l1, solver=liblinear; total time=   0.0

/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 180 is smaller than n_iter=29994. Running 180 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[CV] END ...C=0.0001, max_iter=2500, penalty=l2, solver=saga; total time=   0.0s
[CV] END C=0.0001, max_iter=2500, penalty=l2, solver=liblinear; total time=   0.0s
[CV] END C=0.0001, max_iter=2500, penalty=l2, solver=liblinear; total time=   0.0s
[CV] END C=0.0001, max_iter=2500, penalty=l2, solver=liblinear; total time=   0.0s
[CV] END C=0.0001, max_iter=2500, penalty=l2, solver=liblinear; total time=   0.0s
[CV] END C=0.0001, max_iter=2500, penalty=l2, solver=liblinear; total time=   0.0s
[CV] END C=0.0001, max_iter=2500, penalty=elasticnet, solver=saga; total time=   0.0s
[CV] END C=0.0001, max_iter=2500, penalty=elasticnet, solver=saga; total time=   0.0s
[CV] END C=0.0001, max_iter=2500, penalty=elasticnet, solver=saga; total time=   0.0s
[CV] END C=0.0001, max_iter=2500, penalty=elasticnet, solver=saga; total time=   0.0s
[CV] END C=0.0001, max_iter=2500, penalty=elasticnet, solver=saga; total time=   0.0s
[CV] END C=0.0001, max_iter=2500, penalty=elasticnet, solver=liblinear; to

/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END C=21.54434690031882, max_iter=1000, penalty=l1, solver=saga; total time=   0.7s


...

/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END ..C=10000.0, max_iter=1000, penalty=l2, solver=saga; total time=  16.0s
[CV] END ..C=10000.0, max_iter=1000, penalty=l2, solver=saga; total time=  15.5s
[CV] END ..C=10000.0, max_iter=1000, penalty=l2, solver=saga; total time=  15.0s
[CV] END ..C=10000.0, max_iter=1000, penalty=l2, solver=saga; total time=  15.3s
[CV] END ..C=10000.0, max_iter=1000, penalty=l2, solver=saga; total time=  14.4s
[CV] END C=10000.0, max_iter=1000, penalty=l2, solver=liblinear; total time=   0.0s
[CV] END C=10000.0, max_iter=1000, penalty=l2, solver=liblinear; total time=   0.0s
[CV] END C=10000.0, max_iter=1000, penalty=l2, solver=liblinear; total time=   0.0s
[CV] END C=10000.0, max_iter=1000, penalty=l2, solver=liblinear; total time=   0.0s
[CV] END C=10000.0, max_iter=1000, penalty=l2, solver=liblinear; total time=   0.0s
[CV] END C=10000.0, max_iter=2500, penalty=l2, solver=liblinear; total time=   0.0s
[CV] END C=10000.0, max_iter=2500, penalty=l2, solver=liblinear; total time=   0.0s
[CV] EN

/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_search.py:1102: UserWarning: One or more of the test scores are non-finite: [0.36666667 0.56666667 0.36666667 0.73333333        nan        nan
 0.36666667 0.56666667 0.36666667 0.73333333        nan        nan
 0.56666667 0.56666667 0.43333333 0.73333333        nan        nan
 0.36666667 0.56666667 0.43333333 0.73333333        nan        nan
 0.7        0.56666667 0.63333333 0.73333333        nan        nan
 0.63333333 0.56666667 0.63333333 0.73333333        nan        nan
 0.63333333 0.56666667 0.7        0.73333333        nan        nan
 0.43333333 0.56666667 0.7        0.73333333        nan        nan
 0.56666667 0.56666667 0.63333333 0.73333333        nan        nan
 0.43333333 0.56666667 0.76666667 0.8               nan        nan
 0.43333333 0.56666667 0.76666667 0.8               nan        nan
 0.36666667 0.56666667 0.76666667 0.8               nan        nan
 0.7        0.56666667 0.76666667 0.7              

[CV] END C=10000.0, max_iter=2500, penalty=l2, solver=liblinear; total time=   0.2s
[CV] END C=10000.0, max_iter=2500, penalty=l2, solver=liblinear; total time=   0.1s
[CV] END C=10000.0, max_iter=2500, penalty=l2, solver=liblinear; total time=   0.1s
[CV] END C=10000.0, max_iter=2500, penalty=l2, solver=liblinear; total time=   0.1s
[CV] END C=10000.0, max_iter=2500, penalty=l2, solver=liblinear; total time=   0.1s
[CV] END C=10000.0, max_iter=5000, penalty=l2, solver=liblinear; total time=   0.2s
[CV] END C=10000.0, max_iter=5000, penalty=l2, solver=liblinear; total time=   0.1s
[CV] END C=10000.0, max_iter=5000, penalty=l2, solver=liblinear; total time=   0.1s
[CV] END C=10000.0, max_iter=5000, penalty=l2, solver=liblinear; total time=   0.1s
[CV] END C=10000.0, max_iter=5000, penalty=l2, solver=liblinear; total time=   0.1s
[CV] END ..C=10000.0, max_iter=5000, penalty=l1, solver=saga; total time= 2.1min
[CV] END ..C=10000.0, max_iter=5000, penalty=l1, solver=saga; total time= 2.2mi

/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_search.py:1102: UserWarning: One or more of the test scores are non-finite: [0.36666667 0.56666667 0.36666667 0.73333333        nan        nan
 0.36666667 0.56666667 0.36666667 0.73333333        nan        nan
 0.56666667 0.56666667 0.43333333 0.73333333        nan        nan
 0.36666667 0.56666667 0.43333333 0.73333333        nan        nan
 0.7        0.56666667 0.63333333 0.73333333        nan        nan
 0.63333333 0.56666667 0.63333333 0.73333333        nan        nan
 0.63333333 0.56666667 0.7        0.73333333        nan        nan
 0.43333333 0.56666667 0.7        0.73333333        nan        nan
 0.56666667 0.56666667 0.63333333 0.73333333        nan        nan
 0.43333333 0.56666667 0.76666667 0.8               nan        nan
 0.43333333 0.56666667 0.76666667 0.8               nan        nan
 0.36666667 0.56666667 0.76666667 0.8               nan        nan
 0.7        0.56666667 0.76666667 0.7              